In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
import os
import seaborn as sns
import sys
sys.path.append('..')
from madule import plots

In [30]:
# This should be the first module to be run

In [31]:
"""Add reversal poins as a new column and then collect all sessions and runs files into one csv file"""

# List of subjects
subList = ['sub-004', 'sub-010', 'sub-012', 'sub-025', 'sub-026', 'sub-029', 'sub-030',
           'sub-033', 'sub-034', 'sub-036', 'sub-040', 'sub-041', 'sub-042', 'sub-044', 
           'sub-045', 'sub-047', 'sub-048', 'sub-052', 'sub-054', 'sub-056', 'sub-059', 
           'sub-060', 'sub-064', 'sub-065', 'sub-067', 'sub-069', 'sub-070', 'sub-071', 
           'sub-074', 'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 
           'sub-081', 'sub-082', 'sub-083', 'sub-085', 'sub-087', 'sub-088', 'sub-089', 
           'sub-090',  'sub-092', 'sub-108', 'sub-109']

# Group labels
randomGroupLabel = pd.read_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/randomGroupLabel.csv')
# Concatenating data
dataAll = pd.DataFrame([])

for subName in subList:
    
    # List of existing .csv files for each session and run realted to the subject
    files = ['/ses-02achieva7t/' + subName + '_ses-02achieva7t_task-DA_run-1_beh',
             '/ses-02achieva7t/' + subName + '_ses-02achieva7t_task-DA_run-2_beh',
             '/ses-03achieva7t/' + subName + '_ses-03achieva7t_task-DA_run-1_beh',
             '/ses-03achieva7t/' + subName + '_ses-03achieva7t_task-DA_run-2_beh']

    # Main directory of the subject
    subMainDirec = '/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/originalfMRIbehFiles/'

    # Making empty Dataframe to be concatenated in all four .csv file of the subject
    data = pd.DataFrame([])
    for i in range(len(files)):
        dirc = subMainDirec + subName + files[i] + '.csv'
        df = pd.read_csv(dirc)
        data = pd.concat([data, df])
    
    # Order of conditiond for all runs and sessions
    blocks = data.groupby(['session', 'run'])['block'].unique().to_numpy()
    blocks = np.array([blocks[0], blocks[1], blocks[2], blocks[3]]).flatten()
  
    # reversal point for session one
    data_reverse_session1 = loadmat(subMainDirec + subName + files[0] + '.mat')
    sess1_blockList1_1 = data_reverse_session1['blockList1_1'][0][0]
    sess1_blockList1_2 = data_reverse_session1['blockList1_2'][0][0]
    sess1_blockList2_1 = data_reverse_session1['blockList2_1'][0][0]
    sess1_blockList2_2 = data_reverse_session1['blockList2_2'][0][0]
    # reversal point for session two
    data_reverse_session2 = loadmat(subMainDirec + subName + files[2] + '.mat')
    sess2_blockList1_1 = data_reverse_session2['blockList1_1'][0][0]
    sess2_blockList1_2 = data_reverse_session2['blockList1_2'][0][0]
    sess2_blockList2_1 = data_reverse_session2['blockList2_1'][0][0]
    sess2_blockList2_2 = data_reverse_session2['blockList2_2'][0][0]
    
    data_reverse = np.array([sess1_blockList1_1, sess1_blockList1_2, sess1_blockList2_1, sess1_blockList2_2,
                             sess2_blockList1_1, sess2_blockList1_2, sess2_blockList2_1, sess2_blockList2_2])
    
    # Add new column to main data 
    data['reverse'] = np.nan
    idx = 0
    for s in range(1, 3):
        for r in range(1, 3):
            for b in range(1, 3):
                data.loc[(data['session']==s)&(data['run']==r)&(data['block']==blocks[idx]), 'reverse'] = data_reverse[idx]
                idx +=1
                
    #save file name
    saveFile = subMainDirec + subName + '/' + subName + '_achieva7t_task-DA_beh.csv'
    data.to_csv(saveFile ,index=False)    
    
    # set the name of participants to each data
    data['sub_ID'] = subName
    # read and then set the number of group
    labelSes1 = int(randomGroupLabel.loc[randomGroupLabel['sub-ID'] == subName, 'ses-02'])
    labelSes2 = int(randomGroupLabel.loc[randomGroupLabel['sub-ID'] == subName, 'ses-03'])
    data.loc[data['session'] == 1, 'group'] = str(labelSes1)
    data.loc[data['session'] == 2, 'group'] = str(labelSes2)
    # Detection of irregular responces (no-responses or error responces)
    temp = data['pushed'].to_numpy().astype(int)
    dataClear = data[temp>=0]
    #save file name
    saveFile = subMainDirec + subName + '/' + subName + '_behData'
    # Concatenating each data
    dataAll = pd.concat([dataAll, dataClear])

IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
dataAll.to_csv('/mnt/projects/7TPD/bids/derivatives/fMRI_DA/data_BehModel/originalfMRIbehFiles/AllBehData/behAll.csv', index=False)